In [4]:
import networkx as nx
import pandas as pd
import tqdm

In [21]:
df = pd.read_table("../data/conceptnet-assertions-5.6.0.csv.gz", compression='gzip', chunksize=1000000, names=['url', 'relation', 'start', 'end', 'data'])

G = nx.DiGraph()

for chunk in df:    
    for row in tqdm.tqdm(chunk.itertuples(), total=1000000):
        start = row.start
        end = row.end
        rel = row.relation
        
        if not start.startswith("/c/en") or not end.startswith("/c/en"):
            continue
        
        if rel == '/r/IsA':
            G.add_edge(start, end)

 76%|███████▌  | 755210/1000000 [00:01<00:00, 681952.56it/s]


In [22]:
nx.write_gexf(G, "../data/conceptnet.gexf")

In [23]:
scores = nx.pagerank(G)

In [24]:
from heapq import nlargest

In [38]:
best_nodes = nlargest(iterable=scores, key=scores.get, n=200)
gbest = nx.DiGraph()

for src,dst in G.edges:
    if src not in best_nodes or dst not in best_nodes:
        continue
        
    if scores[src] < scores[dst]:
        gbest.add_edge(src.split('/')[3], dst.split('/')[3])

In [42]:
dot = nx.nx_pydot.to_pydot(gbest)
dot.set_rankdir("LR")
dot.write_pdf("../data/conceptnet.pdf")

In [43]:
nx.write_gexf(gbest, "../data/concepnet-best.gexf")